In [1]:
import requests
import csv

# Super Mario Bros 1 = om1m3625
# Platforms
# NES = jm95z9ol
# Super Nintendo (SNES) = 83exk6l5
# WiiVC = nzelreqp
# GBA = 3167d6q2
# Wii = v06dk3e4
# Nintendo 3DS Virtual Console = 7g6mx89r
# Fanicom Disk System (FDS) = mr6k409z
# Wii U Virtual Console = v06dr394
# NESClassic = kz9wynep
# Switch = 7m6ylw9p
# SNESClassic = 83exovel

# New Super Mario Bros. U = o1y3l46q
# Platform
# Wii U = 8gejn93d


# Super Mario Bros. 3 = l3dx51yv
# Platforms
# NES = jm95z9ol
# SNES = 83exk6l5
# WiiVC = nzelreqp
# Wii = v06dk3e4
# 3DSVC = 7g6mx89r
# WiiUVC = v06dr394
# NESClassic = kz9wynep
# Switch = 7m6ylw9p


# Super Mario World = pd0wq31e
# Platforms
# SNES = 83exk6l5
# WiiVC = nzelreqp
# 3DSVC = 7g6mx89r
# WiiUVC = v06dr394
# Switch = 7m6ylw9p
# SNESClassic = 83exovel


# Super Mario Kart = vo6g8128
# Platforms
# SNES = 83exk6l5
# WiiVC = nzelreqp
# WiiUVC = v06dr394
# New3DSVC = v06do394
# Switch = 7m6ylw9p
# SNESClassic = 83exovel

# Mario Kart Wii = l3dxogdy
# Platforms
# Wii U = 8gejn93d
# Wii = v06dk3e4

def platformName(platformID):
    if platformID == 'jm95z9ol':
        return 'NES'
    elif platformID == '83exk6l5':
        return 'SNES'
    elif platformID == 'nzelreqp':
        return 'WiiVC'
    elif platformID == '3167d6q2':
        return 'GBA'
    elif platformID == 'v06dk3e4':
        return 'Wii'
    elif platformID == '7g6mx89r':
        return '3DSVC'
    elif platformID == 'mr6k409z':
        return 'FDS'
    elif platformID == 'v06dr394':
        return 'WiiUVC'
    elif platformID == 'kz9wynep':
        return 'NESClassic'
    elif platformID == '7m6ylw9p':
        return 'Switch'
    elif platformID == '83exovel':
        return 'SNESClassic'
    elif platformID == '8gejn93d':
        return 'WiiU'
    elif platformID == 'v06do394':
        return 'New3DSVC'
    else:
        return 'Unknown'

gameIDs = ['pd0wq31e']
urlBase = 'https://www.speedrun.com/api/v1/runs?game='
gameNameURLBase = 'https://www.speedrun.com/api/v1/games/'
columnNames = [ "Run ID", "User ID", "User Name", "Hours", "Minutes", "Seconds", "Platform ID", "Platform Name", "Verified", "URL", "Date"]

for x in range(0,len(gameIDs)):
    url = urlBase + gameIDs[x]
    gameURL = gameNameURLBase + gameIDs[x]
    gameNameR = requests.get(gameURL, timeout=(3.05,27))
    gameNameJSON = gameNameR.json()
    gameName = gameNameJSON['data']['names']['international']
    fileName = gameName.replace(" ", "") + ".csv"

    
    with open(fileName, 'w', newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(columnNames)
    
        i = 20
        r = requests.get(url, timeout=(3.05, 27))
        ourJSON = r.json()
        while ourJSON['data'] != []:
            # do our data pull
            for y in range(0, len(ourJSON['data'])):
                toWrite = []
                toWrite.append(ourJSON['data'][y]['id'])
                if 'id' in ourJSON['data'][y]['players'][0]:
                    userID = ourJSON['data'][y]['players'][0]['id']
                    toWrite.append(userID)
                else:
                    userID = ourJSON['data'][y]['players'][0]['name']
                    toWrite.append('ERROR')
                userURL = ourJSON['data'][y]['players'][0]['uri']
                p = requests.get(userURL, timeout=(3.05,27))
                userJSON = p.json()
                if 'names' in userJSON['data']:
                    toWrite.append(userJSON['data']['names']['international'])
                else:
                    toWrite.append(userID)

                # time

                time = ourJSON['data'][y]['times']['primary']
                # removes PT from playtime
                time = time[2:]
                hour, h, remainder = time.partition('H')
                if hour.isdigit():
                    toWrite.append(float(hour))
                    time = remainder
                else:
                    toWrite.append(0)
                    time = hour
                minute, m, remainder = time.partition('M')
                if minute.isdigit():
                    toWrite.append(float(minute))
                    time = remainder
                else:
                    toWrite.append(0)
                    time = minute
                remainTime, s ,remainder = time.partition('S')
                second, dot, millis = remainTime.partition('.')
                if second.isdigit():
                    if millis.isdigit():
                        toWrite.append(float(second)+(float(millis)/(10**(len(millis)))))
                    else:
                        toWrite.append(float(second))
                else:
                    toWrite.append(0)

                # platform

                platformID = ourJSON['data'][y]['system']['platform']
                toWrite.append(platformID)
                toWrite.append(platformName(platformID))

                toWrite.append(ourJSON['data'][y]['status']['status'])

                toWrite.append(ourJSON['data'][y]['links'][0]['uri'])

                toWrite.append(ourJSON['data'][y]['date'])

                wr.writerow(toWrite)
                    

            # get next page
            newURL = url + '&offset=' + str(i)
            r = requests.get(newURL, timeout=(3.05,27))
            ourJSON = r.json()
            i = i + 20
            #ourJSON['data'] = [] 
        

#with open('runs.csv', 'w', newline='') as myfile:
#    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
#    wr.writerow(mylist2)


UnicodeEncodeError: 'charmap' codec can't encode characters in position 16-20: character maps to <undefined>